In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import sklearn

In [2]:
import pandas_datareader as web

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [4]:
df = web.DataReader('aapl', "yahoo")

In [88]:
def create_dataset(stock_symbol, start_date, end_date, lags=5):
    
    df = web.DataReader(stock_symbol, "yahoo", start_date, end_date)
    
    dflag = pd.DataFrame(index=df.index)
    dflag["Today"] = df["Adj Close"]
    dflag["Volume"] = df["Volume"]    
    
    for i in range(0, lags):
        dflag["Lag{}".format(str(i+1))] = df["Adj Close"].shift(i+1)
    
    dfret = pd.DataFrame(index=dflag.index)
    dfret["Volume"] = dflag["Volume"]
    dfret["Today"] = dflag["Today"].pct_change()*100
    
    for i in range(0, lags):
        dfret["Lag{}".format(str(i+1))] = dflag["Lag{}".format(str(i+1))].pct_change()*100    
        
    dfret['Direction'] = np.sign(dfret["Today"])
    
    dfret.drop(dfret.index[:lags+1], inplace=True)
    
    dfret.drop(dfret[dfret["Direction"]==0.0].index, inplace=True)
    return dfret
    

In [101]:
data = create_dataset("AAPL", datetime(2012, 1, 1), datetime(2018, 5, 31), 5)
data.head()

,Volume,Today,Lag1,Lag2,Lag3,Lag4,Lag5,Direction
Date,,,,,,,,
2012-01-11,53771200.0,-0.163016,0.358024,-0.158597,1.045358,1.110186,0.537446,-1.0
2012-01-12,53146800.0,-0.274536,-0.163016,0.358024,-0.158597,1.045358,1.110186,-1.0
2012-01-13,56505400.0,-0.374936,-0.274536,-0.163016,0.358024,-0.158597,1.045358,-1.0
2012-01-17,60724300.0,1.164809,-0.374936,-0.274536,-0.163016,0.358024,-0.158597,1.0
2012-01-18,69197800.0,1.038395,1.164809,-0.374936,-0.274536,-0.163016,0.358024,1.0


In [102]:
data["Direction"].value_counts()

 1.0    834
-1.0    770
Name: Direction, dtype: int64

In [103]:
X = data[["Lag1", "Lag2", "Lag3", "Lag4", "Lag5"]]
y = data["Direction"]

In [104]:
start_test = datetime(2017,1,1)

In [105]:
X_train = X[X.index<start_test]
y_train = y[y.index<start_test]
X_test = X[X.index>=start_test]
y_test = y[y.index>=start_test]

In [106]:
model = LogisticRegression()

In [107]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [108]:
pred = model.predict(X_test)

In [109]:
print("Accuracy of logistic regression model: {}".format(model.score(X_test, y_test)))

Accuracy of logistic regression model: 0.53954802259887


In [110]:
confusion_matrix(pred, y_test)

array([[ 30,  29],
       [134, 161]])

In [111]:
print("Confussion matrix: {}".format(confusion_matrix(pred, y_test)))

Confussion matrix: [[ 30  29]
 [134 161]]
